In [1]:
import pandas as pd
import os
import random
import time
import seaborn as sns
import matplotlib.pyplot as plt

In [31]:
data_dir = '/opt/ml/input/data/'
csv_file_path = os.path.join(data_dir, 'train_data_test_fe.csv')
df = pd.read_csv(csv_file_path)
df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,year,month,day,hour,...,t_type_ans_mean,t_type_ans_sum,t_type_ans_std,t_type_time_mean,t_type_time_std,test_lvl,test_lvl_cat,test_type_lvl,test_type_lvl_cat,test_total_answer
0,0,A060001001,A060000001,1,1585009031,7224,2020,3,24,0,...,0.708948,186888,0.454248,55.175012,70.312176,23.877991,0,77.826561,1,0
1,0,A060001002,A060000001,1,1585009034,7225,2020,3,24,0,...,0.708948,186888,0.454248,55.175012,70.312176,23.877991,0,77.826561,1,1
2,0,A060001003,A060000001,1,1585009042,7225,2020,3,24,0,...,0.708948,186888,0.454248,55.175012,70.312176,23.877991,0,77.826561,1,2
3,0,A060001004,A060000001,1,1585009049,7225,2020,3,24,0,...,0.708948,186888,0.454248,55.175012,70.312176,23.877991,0,77.826561,1,3
4,0,A060001005,A060000001,1,1585009056,7225,2020,3,24,0,...,0.708948,186888,0.454248,55.175012,70.312176,23.877991,0,77.826561,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,1591339821,438,2020,6,5,6,...,0.702207,191579,0.457289,52.177665,66.330204,76.231909,4,74.305218,0,1389
2266582,7441,A040165001,A040000165,1,1597971999,8836,2020,8,21,1,...,0.683727,182088,0.465022,52.161823,66.587362,71.450640,4,76.290476,1,1092
2266583,7441,A040165002,A040000165,1,1597972010,8836,2020,8,21,1,...,0.683727,182088,0.465022,52.161823,66.587362,71.450640,4,76.290476,1,1093
2266584,7441,A040165003,A040000165,1,1597972056,8836,2020,8,21,1,...,0.683727,182088,0.465022,52.161823,66.587362,71.450640,4,76.290476,1,1094


In [32]:
def tag_fe(df):
    ## tag의 평균 정답률, 정답 총합, 표준편차
    correct_k = df[df['userID'].shift(1) == df['userID']].groupby(['KnowledgeTag'])['answerCode'].agg(['mean', 'sum','std'])
    correct_k.columns = ["tag_ans_mean", 'tag_ans_sum','tag_ans_std']
    df = pd.merge(df, correct_k, on=['KnowledgeTag'], how="left")
    
    ## 태그의 평균 풀이 시간, 표준편차
    time_k = df.groupby(['KnowledgeTag'])['time'].agg(['mean', 'std'])
    time_k.columns = ["tag_time_mean", 'tag_time_std']
    df = pd.merge(df, time_k, on=['KnowledgeTag'], how="left")

    ## 태그 난이도
    df['tag_lvl'] = df['tag_time_mean'] / df['tag_ans_mean']

    # 범주화
    tag_cat_num = 10
    df['tag_lvl_cat'] = pd.qcut(df['tag_lvl'],tag_cat_num,labels=[i for i in range(tag_cat_num)])

    ## 태그 노출 횟수
    df['tag_total_answer'] = df.groupby('KnowledgeTag')['answerCode'].cumcount()
    
    return df

In [33]:
df = tag_fe(df)

In [34]:
df[df['userID'].shift(1) != df['userID']]

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,year,month,day,hour,...,test_type_lvl_cat,test_total_answer,tag_ans_mean,tag_ans_sum,tag_ans_std,tag_time_mean,tag_time_std,tag_lvl,tag_lvl_cat,tag_total_answer
0,0,A060001001,A060000001,1,1585009031,7224,2020,3,24,0,...,1,0,0.942913,479,0.232237,15.587706,32.699735,16.531429,0,0
745,1,A040013001,A040000013,1,1578300043,2048,2020,1,6,8,...,1,0,0.620734,964,0.485361,68.010165,71.579758,109.564094,8,0
1678,2,A030050001,A030000050,1,1578654173,407,2020,1,10,11,...,0,0,0.766961,3301,0.422816,46.231393,57.179329,60.278678,2,0
1954,5,A080001001,A080000001,1,1581537620,4604,2020,2,12,20,...,1,0,0.487674,455,0.500116,29.347584,44.414370,60.178672,2,0
2787,6,A030016001,A030000016,1,1583750908,7307,2020,3,9,10,...,0,0,0.849312,3455,0.357789,46.216456,55.136546,54.416366,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266515,7436,A050095001,A050000095,1,1583217245,360,2020,3,3,6,...,2,908,0.788977,3178,0.408085,41.333912,54.765753,52.389237,1,4030
2266530,7437,A040072001,A040000072,1,1585035915,2076,2020,3,24,7,...,1,1590,0.793621,3384,0.404753,35.597991,48.357023,44.855152,0,4275
2266546,7438,A080002001,A080000002,1,1582664394,4605,2020,2,25,20,...,1,1326,0.823529,126,0.382472,63.612613,72.888953,77.243887,4,221
2266562,7440,A050096001,A050000096,1,1597813039,5267,2020,8,19,4,...,2,1120,0.783355,1233,0.412090,48.471889,61.246009,61.877334,2,1580


In [35]:
train_df = df[df['userID'].shift(1) != df['userID']]
tdf = pd.read_csv(csv_file_path) 
tdf.sort_values(by=['userID','Timestamp'], inplace=True)
tdf.loc[tdf['userID'].shift(1) != tdf['userID'],'answerCode'] = -1
tdf = tag_fe(tdf)
test_df = tdf[tdf['userID'].shift(1) != tdf['userID']]
diff =pd.concat([train_df,test_df],axis=1)
diff = diff.T.drop_duplicates(keep=False).T
diff = diff.drop(['answerCode'],axis=1)
diff

""
0
745
1678
1954
2787
...
2266515
2266530
2266546
2266562


In [36]:
df
df.to_csv('/opt/ml/input/data/train_data_tag_fe.csv',index=False)